# Run the pyWOMBAT biogeochemical model in 1D water column configuration

In [9]:
import sys
import os
import itertools
import logging
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
import scipy as sci
from scipy.stats import qmc
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import multiprocessing

# Ensure we are in the correct directory
os.chdir("/home/581/pjb581/py-WOMBAT/lite")
print(os.getcwd())
from main import main  # Import the main function from main.py

# Define output directory and ensure it exists (MAKE SURE A FORWARD SLASH EXISTS AT END)
OUTPUT_DIR = "/g/data/es60/pjb581/py-WOMBAT/output/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("python version =",sys.version[:5])
print("numpy version =", np.__version__)
print("xarray version =", xr.__version__)
print("scipy version =", sci.__version__)
print("PyCO2SYS version =", pyco2.__version__)

print(datetime.now())

"""
NOTE: Need read permissions for groups es60, gb6, qv56
"""

/home/581/pjb581/py-WOMBAT/lite
python version = 3.10.
numpy version = 1.24.4
xarray version = 2023.8.0
scipy version = 1.15.1
PyCO2SYS version = 1.8.3.4
2025-04-25 15:09:20.607596


'\nNOTE: Need read permissions for groups es60, gb6, qv56\n'

In [10]:
!git branch

  master
  pyWOMBAT-on-Gadi
* pyWOMBAT-on-Gadi_microbes


## Define the experimental parameters

In [11]:
year = 2001
days = 365*1
lon = 340.0
lat = 30.0
atm_co2 = 400.0

info = np.array([[1.0, year, days, 340.0, 30.0, atm_co2],
                 [2.0, year, days, 200.0, 10.0, atm_co2]])
info

array([[1.000e+00, 2.001e+03, 3.650e+02, 3.400e+02, 3.000e+01, 4.000e+02],
       [2.000e+00, 2.001e+03, 3.650e+02, 2.000e+02, 1.000e+01, 4.000e+02]])

## Save parameter sets in excel

In [12]:
names = ["expnum", "year", "days", "longitude", "latitude", "atmCO2"]

paramsets = pd.DataFrame(info, columns=names)
paramsets

,expnum,year,days,longitude,latitude,atmCO2
0,1.0,2001.0,365.0,340.0,30.0,400.0
1,2.0,2001.0,365.0,200.0,10.0,400.0


## Run the experiments

In [13]:
# setup the log file for each experiment
def setup_logging():
    log_file = f"experiment_output_{multiprocessing.current_process().pid}.log"
    logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")


# Define function to run one experiment
def run_experiment(exp):

    setup_logging()
    
    expnum, yr, dayl, lon, lat, atm_co2 = exp
    
    logging.info(f"\n🚀 Running Experiment with Params: {exp}")
    logging.info(f"Process {multiprocessing.current_process().pid} started at {datetime.now()}")
    
    # Run the main function with these parameters
    main(expnum, yr, dayl, lon, lat, atm_co2)

    logging.info(f"Process {multiprocessing.current_process().pid} finished at {datetime.now()}")
    logging.info(f"✅ Experiment Complete: {exp}")
    


In [14]:
### Cut out the experiments already run
done = set()
for fn in os.listdir(OUTPUT_DIR):
    if "exp" in fn and fn.endswith(".nc"):
            try:
                # Extract the experiment number
                exp_str = fn.split("exp")[-1].split(".nc")[0]
                expnum = int(exp_str)
                done.add(expnum)
            except ValueError:
                # In case conversion fails, skip this file
                continue

todo = [exp for exp in info if exp[0] not in done]
np.shape(todo)

(2, 6)

In [ ]:
# Run experiments in parallel using multiprocessing
if __name__ == "__main__":
    num_processes = min(min(multiprocessing.cpu_count(), 2), len(todo))  # Use at most the available cores to a max of 16
    print("Running %i parallel experiments"%(num_processes))
    with multiprocessing.Pool(processes=num_processes) as pool:
        pool.map(run_experiment, todo)

    # Check generated output files
    output_files = os.listdir("output")
    print("\nGenerated output files:", output_files)
    

Running 2 parallel experiments
